In [ ]:
! pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Audio.wav Dosyası Kopyalama

In [ ]:
import os
import shutil

base_input_path = '/content/gdrive/MyDrive/wav2lip_scaled_dataset_drive/'
base_output_path = '/content/gdrive/MyDrive/wav2lip_rescaled_dataset_drive/'


for klasor_id_1 in os.listdir(base_input_path):
    klasor_input_path_1 = os.path.join(base_input_path, klasor_id_1)

    if not os.path.isdir(klasor_input_path_1):
        print(f"Skipping {klasor_input_path_1} as it is not a directory")
        continue

    for klasor_id_2 in os.listdir(klasor_input_path_1):
        klasor_input_path_2 = os.path.join(klasor_input_path_1, klasor_id_2)

        if not os.path.isdir(klasor_input_path_2):
            print(f"Skipping {klasor_input_path_2} as it is not a directory")
            continue

        klasor_output_path = os.path.join(base_output_path, klasor_id_1, klasor_id_2)

        #os.makedirs(klasor_output_path, exist_ok=True)

        input_audio_path = os.path.join(klasor_input_path_2, 'audio.wav')
        output_audio_path = os.path.join(klasor_output_path, 'audio.wav')

        if os.path.isfile(input_audio_path):
            try:
                shutil.copy(input_audio_path, output_audio_path)
                print(f'Copied {input_audio_path} to {output_audio_path}')
            except Exception as e:
                print(f"Error copying {input_audio_path}: {e}")
        else:
            print(f"No audio.wav found in {klasor_input_path_2}")


#ESRGAN Upscale

In [ ]:
import os
import torch
from PIL import Image
from RealESRGAN import RealESRGAN


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)


base_input_path = '/content/gdrive/MyDrive/wav2lip_dataset_drive/'
base_output_path = '/content/gdrive/MyDrive/wav2lip_scaled_dataset_drive/'


for klasor_id_1 in os.listdir(base_input_path):
    klasor_input_path_1 = os.path.join(base_input_path, klasor_id_1)

    if not os.path.isdir(klasor_input_path_1):
        print(f"Skipping {klasor_input_path_1} as it is not a directory")
        continue

    for klasor_id_2 in os.listdir(klasor_input_path_1):
        klasor_input_path_2 = os.path.join(klasor_input_path_1, klasor_id_2)

        if not os.path.isdir(klasor_input_path_2):
            print(f"Skipping {klasor_input_path_2} as it is not a directory")
            continue

        klasor_output_path = os.path.join(base_output_path, klasor_id_1, klasor_id_2)
        os.makedirs(klasor_output_path, exist_ok=True)

        for img_id in os.listdir(klasor_input_path_2):
            if img_id.endswith('.jpg'):
                input_image_path = os.path.join(klasor_input_path_2, img_id)
                output_image_path = os.path.join(klasor_output_path, img_id)

                try:
                    image = Image.open(input_image_path).convert('RGB')
                    sr_image = model.predict(image)
                    sr_image.save(output_image_path)
                    print(f'Scaled image saved to {output_image_path}')
                except Exception as e:
                    print(f"Error processing {input_image_path}: {e}")


#OpenCV Resize

In [ ]:
import os
import cv2

base_input_path = '/content/gdrive/MyDrive/wav2lip_scaled_dataset_drive/'
base_output_path = '/content/gdrive/MyDrive/wav2lip_rescaled_dataset_drive/'


target_size = (440,440)

for klasor_id_1 in os.listdir(base_input_path):
    klasor_input_path_1 = os.path.join(base_input_path, klasor_id_1)

    if not os.path.isdir(klasor_input_path_1):
        print(f"Skipping {klasor_input_path_1} as it is not a directory")
        continue

    for klasor_id_2 in os.listdir(klasor_input_path_1):
        klasor_input_path_2 = os.path.join(klasor_input_path_1, klasor_id_2)

        if not os.path.isdir(klasor_input_path_2):
            print(f"Skipping {klasor_input_path_2} as it is not a directory")
            continue

        klasor_output_path = os.path.join(base_output_path, klasor_id_1, klasor_id_2)
        os.makedirs(klasor_output_path, exist_ok=True)

        for img_id in os.listdir(klasor_input_path_2):
            if img_id.endswith('.jpg'):
                input_image_path = os.path.join(klasor_input_path_2, img_id)
                output_image_path = os.path.join(klasor_output_path, img_id)

                try:
                    image = cv2.imread(input_image_path)
                    resized_image = cv2.resize(image, target_size)
                    cv2.imwrite(output_image_path, resized_image)
                    print(f'Resized image saved to {output_image_path}')
                except Exception as e:
                    print(f"Error processing {input_image_path}: {e}")


#Train.txt & Valid.txt olusturma

In [ ]:
import os
from glob import glob
from sklearn.model_selection import train_test_split

def trained_data_name_format(base_path):
    result = list(glob("{}/*".format(base_path)))
    result_list = []

    for i, dirpath in enumerate(result):
        dirs = os.listdir(dirpath)
        for f in dirs:
            result_list.append(str(os.path.join(dirpath.replace(base_path, ''), f)))

    if len(result_list) < 14:
        test_result = val_result = train_result = result_list
    else:
        train_result, test_result = train_test_split(result_list, test_size=0.2, random_state=42)
        test_result, val_result = train_test_split(test_result, test_size=0.5, random_state=42)

    os.makedirs("filelists", exist_ok=True)
    for file_name, dataset in zip(("train.txt", "test.txt", "val.txt"), (train_result, test_result, val_result)):
        with open(os.path.join("filelists", file_name), 'w', encoding='utf-8') as fi:
            fi.write("\n".join(dataset))

trained_data_name_format("/content/gdrive/MyDrive/wav2lip_rescaled_dataset_drive/")